# **Convolutional Neural Networks (CNNs)**

In this section, we are going to discuss about yet powerful, famous model - Convolutional Neural Networks (CNNs). <br>
CNNs are primarily used for image recognition and processing, due to its ability to recognize patterns in the images. <br>
It is a powerful tool but requires a lot of (usually in millions) labelled training data points for effective training of the model. <br>
Here, let's train a CNN for CIFAR10 dataset

In [29]:
# Import necessary modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers 
from tensorflow.keras.datasets import cifar10

In [13]:
# Loading the dataset
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

In [14]:
# Normalizing: Converting 64bits to 32bits for computational efficiency 
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Note that here the array wasn't flattened - CNNs take input in it's original shape

In [23]:
# Building the model
model = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)), # 32x32  - 3 channels RGB
        layers.Conv2D(32, (3,3), padding='valid', activation='relu'),  # padding='same' -> same shape;  padding='valid' -> shape changes;
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64,3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.Flatten(),
        layers.Dense(10)
    ]
)

#model.summary()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits =True),
    optimizer = keras.optimizers.Adam(3e-4),
    metrics = ['accuracy'],
)
print(X_train.shape, y_train.dtype)
#model.summary()
model.fit(X_train, y_train, batch_size=64, epochs=10)
model.evaluate(X_test, y_test, batch_size=64)

(50000, 32, 32, 3) uint8
Epoch 1/10
782/782 [==============================] - 22s 26ms/step - loss: 1.6769 - accuracy: 0.3911
Epoch 2/10
782/782 [==============================] - 23s 30ms/step - loss: 1.3609 - accuracy: 0.5173
Epoch 3/10
782/782 [==============================] - 23s 30ms/step - loss: 1.2261 - accuracy: 0.5686
Epoch 4/10
782/782 [==============================] - 24s 30ms/step - loss: 1.1304 - accuracy: 0.6062
Epoch 5/10
782/782 [==============================] - 24s 30ms/step - loss: 1.0659 - accuracy: 0.6297
Epoch 6/10
782/782 [==============================] - 25s 31ms/step - loss: 1.0096 - accuracy: 0.6525
Epoch 7/10
782/782 [==============================] - 24s 30ms/step - loss: 0.9605 - accuracy: 0.6693
Epoch 8/10
782/782 [==============================] - 24s 31ms/step - loss: 0.9216 - accuracy: 0.6825
Epoch 9/10
782/782 [==============================] - 23s 29ms/step - loss: 0.8861 - accuracy: 0.6944
Epoch 10/10
157/157 [==============================] - 2s

[0.9641452431678772, 0.6675000190734863]

In [26]:
# Functional API
def build_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32,3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,5,padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(128,3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = build_model()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(3e-4),
    metrics = ['accuracy']
)
model.fit(X_train, y_train, batch_size=64, epochs=10)
model.evaluate(X_test, y_test, batch_size=64)

Epoch 1/10
782/782 [==============================] - 78s 96ms/step - loss: 1.2875 - accuracy: 0.5459
Epoch 2/10
782/782 [==============================] - 81s 103ms/step - loss: 0.8924 - accuracy: 0.6887
Epoch 3/10
782/782 [==============================] - 85s 109ms/step - loss: 0.7271 - accuracy: 0.7479
Epoch 4/10
782/782 [==============================] - 88s 113ms/step - loss: 0.6134 - accuracy: 0.7870
Epoch 5/10
782/782 [==============================] - 84s 108ms/step - loss: 0.5155 - accuracy: 0.8226
Epoch 6/10
782/782 [==============================] - 85s 108ms/step - loss: 0.4355 - accuracy: 0.8510
Epoch 7/10
782/782 [==============================] - 90s 115ms/step - loss: 0.3613 - accuracy: 0.8774
Epoch 8/10
782/782 [==============================] - 87s 112ms/step - loss: 0.2932 - accuracy: 0.9021
Epoch 9/10
782/782 [==============================] - 78s 100ms/step - loss: 0.2414 - accuracy: 0.9203
Epoch 10/10
157/157 [==============================] - 6s 33ms/step - loss

[1.028511881828308, 0.7106999754905701]

Here as we can see, the training accuracy is too good.. but accuracy on testing data is not that good. This means, the model is overfitting to the training data. So regularization methods should be employed. These regularizations should reduce the gap between training accuracy and the evaluation accuracy. <br>

**Regularization Techniques - Reducing overfitting** <br>
  - Reduce the capacity of model
  - L2 Regularization
  - Dropout
  - Early Stoppage
  - Data Augmentation

In [30]:
def build_regularized_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32,3, padding='same', kernel_regularizer = regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x)    # BatchNorm -> Faster Traning  +  Regularization effect
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,5,padding='same', kernel_regularizer = regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(128,3, padding='same', kernel_regularizer = regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Using regularization like DropOut leads to longer training time
# Drops a lot of connections
model = build_regularized_model()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(3e-4),
    metrics = ['accuracy']
)
model.fit(X_train, y_train, batch_size=64, epochs=150)
model.evaluate(X_test, y_test, batch_size=64)

Epoch 1/150
782/782 [==============================] - 100s 126ms/step - loss: 3.0553 - accuracy: 0.1252
Epoch 2/150
782/782 [==============================] - 109s 140ms/step - loss: 2.3029 - accuracy: 0.1411
Epoch 3/150
782/782 [==============================] - 102s 131ms/step - loss: 2.2356 - accuracy: 0.1462
Epoch 4/150
782/782 [==============================] - 98s 126ms/step - loss: 2.2188 - accuracy: 0.1499
Epoch 5/150
782/782 [==============================] - 100s 128ms/step - loss: 2.2086 - accuracy: 0.1506
Epoch 6/150
782/782 [==============================] - 97s 124ms/step - loss: 2.2016 - accuracy: 0.1499
Epoch 7/150
782/782 [==============================] - 94s 120ms/step - loss: 2.1980 - accuracy: 0.1530
Epoch 8/150
782/782 [==============================] - 103s 131ms/step - loss: 2.1977 - accuracy: 0.1524
Epoch 9/150
782/782 [==============================] - 102s 131ms/step - loss: 2.1895 - accuracy: 0.1552
Epoch 10/150
782/782 [==============================] - 13

[2.0633604526519775, 0.21789999306201935]